In [12]:
! pip3 install evaluate
! pip3 install rouge_score
import os
from torch.utils.data import TensorDataset

try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir(next((root for root, _, files in os.walk("..") if "dsait4090_project_location" in files), ".."))
    print(f'Google Colab: {os.getcwd()}')
except ImportError:
    print(f'Local: {os.getcwd()}')

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1ec405d186b7a7f18ee376f5a410aac5250cacdd3ddeae3fb00dad486115ae15
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Colab: /content/drive/Othercomputers/macbook_pro/fact-checking


In [ ]:
# !nvidia-smi

zsh:1: command not found: nvidia-smi


In [5]:
import os
import pandas as pd
from src.common import DATA_PATH, read_data, save_data

LOCAL_DATA_PATH = os.path.join(DATA_PATH, 'custom_decomposition', 'comparison_strategyqa')

train = read_data(os.path.join(LOCAL_DATA_PATH, 'train.json'))
val = read_data(os.path.join(LOCAL_DATA_PATH, 'val.json'))
test = read_data(os.path.join(LOCAL_DATA_PATH, 'test.json'))

# len(train), len(dev)

# p = int(len(train)*0.9)
# test = train[p:]
# train = train[:p]

print(len(train), len(val), len(test))

def process(data):
    new_data = []
    for p in data:
        new_data.append({
            'claim': p['question'],
            'subquestions': '\n'.join(p['decomposition'])
        })

    return new_data

# train = process(train)
# val = process(val)
# test = process(test)

# def save_csv(path, data):
#     df = pd.DataFrame(data)
#     df.to_csv(path, index=True)

# save_csv(os.path.join(LOCAL_DATA_PATH, 'NEW_train.csv'), train)
# save_csv(os.path.join(LOCAL_DATA_PATH, 'NEW_val.csv'), val)
# save_csv(os.path.join(LOCAL_DATA_PATH, 'NEW_test.csv'), test)

1854 229 207


In [2]:
from evaluate import load
#rouge_metric = load('rouge')

In [3]:
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
# import pytorch_lightning as pl
import torch
# from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse
import datetime

import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from IPython.display import display, HTML
import random
import nltk  # Here to have a nice missing dependency error message early on
import numpy as np
from datasets import load_dataset#, load_metric

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.file_utils import is_offline_mode
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from src.common import DATA_PATH, MODELS_PATH, read_data

nltk.download('punkt')

import wandb
wandb.login()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
ERROR:wandb.sdk.lib.gitlib:git root error: Cmd('git') failed due to: exit code(128)
  cmdline: git rev-parse --show-toplevel
  stderr: 'fatal: not a git repository: /content/drive/Othercomputers/macbook_pro/fact-checking/../.git/modules/fact-checking'
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### References: [Official](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/summarization.ipynb#scrollTo=vc0BSBLIIrJQ), [Community](https://colab.research.google.com/github/elsanns/xai-nlp-notebooks/blob/master/fine_tune_bart_summarization_two_langs.ipynb#scrollTo=6R9d7ELIpX9F)

In [88]:
# BASE_PATH = "/content/drive/MyDrive/DSAIT4090_FinalProject/code/claim-decomp/"
BASE_PATH = LOCAL_DATA_PATH

MODEL_PATH = "google/flan-t5-base"
TOKENIZER_PATH = "google/flan-t5-base"
SAVE_PATH = BASE_PATH + "/test/"
LOGGING_PATH = BASE_PATH + "/test_logs/"
SAVE_MODEL_PATH = BASE_PATH + "/model/"

In [7]:
import pandas as pd
train = pd.read_csv(BASE_PATH + "/train.csv")
val = pd.read_csv(BASE_PATH + "/val.csv")
test = pd.read_csv(BASE_PATH + "/test.csv")
train

,Unnamed: 0,claim,subquestions
0,0,Are more people today related to Genghis Khan ...,How many kids did Julius Caesar have?\nHow man...
1,1,Could the members of The Police perform lawful...,Who can perform lawful arrests?\nAre members o...
2,2,Would a Monoamine Oxidase candy bar cheer up a...,Depression is caused by low levels of what che...
3,3,Would a dog respond to bell before Grey seal?,How sensitive is a grey seal's hearing on land...
4,4,Is a pound sterling valuable?,What is the value of the Pound Sterling based ...
...,...,...,...
1849,1849,Do placozoa get learning disabilities?,What bodily system do learning disabilities af...
1850,1850,Are multiple Christmas Carol's named after Sai...,Who was Christmas carol 'Good King Wenceslas' ...
1851,1851,Are thetan levels found in the New Testament?,Which religion are thetan levels related to?\n...
1852,1852,Would a honey badger's dentures be different f...,What subfamily does the honey badger belong to...


In [ ]:
# data_subq["subquestions"] = data_subq["subquestions"].apply(lambda x: x.replace("[","").replace("]",""))
# data_subq

In [ ]:
# from sklearn.model_selection import train_test_split
# train,val = train_test_split(data_subq,test_size=0.10,random_state=42)
# train

,question,answer,subquestions
1056,Did Pink Floyd have a song about the French Ri...,True,Question 0: What songs by Pink Floyd are named...
2085,Did Moon Jae-in's residence exist when the Wor...,False,Question 0: Where does Moon Jae-in live?\nAnsw...
637,Did the death of Helen Palmer have a significa...,True,Question 0: What relatives did Helen Palmer ha...
48,Is Menthol associated with Thanksgiving?,False,Question 0: What are some common dishes served...
509,Is Capricorn the hypothetical zodiac sign of S...,True,Question 0: What animal represents the zodiac ...
...,...,...,...
1638,Is Cookie Monster's diet Paleo friendly?,False,Question 0: What is the major component of the...
1095,Is LG Electronics located in a city with an of...,True,Question 0: Which city is LG Electronics headq...
1130,Would it be difficult to snowboard on Venus?,True,Question 0: What kind of surface is suitable f...
1294,Can a minor replicate the double-slit experiment?,True,Question 0: What species of living things does...


In [ ]:
# train.to_csv("train_strategyq_with_answer.csv",index=False)
# val.to_csv("val_strategyq_with_answer.csv",index=False)

# Loading and processing the data

In [8]:
TRAIN_FILE_PATH = BASE_PATH + "/train.csv"
VAL_FILE_PATH = BASE_PATH + "/val.csv"
TEST_FILE_PATH = BASE_PATH + "/test.csv"

In [9]:
data_files = {}
data_files["train"] = TRAIN_FILE_PATH
data_files["validation"] = VAL_FILE_PATH
data_files["test"] = TEST_FILE_PATH

In [13]:
xmetric = load('rouge')#load_metric("rouge")
data_files
raw_datasets = load_dataset("csv", data_files=data_files)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [14]:
metric = load('rouge')#load_metric("rouge")


In [15]:
raw_datasets["train"][0]

{'Unnamed: 0': 0,
 'claim': 'Are more people today related to Genghis Khan than Julius Caesar?',
 'subquestions': 'How many kids did Julius Caesar have?\nHow many kids did Genghis Khan have?\nIs #2 greater than #1?'}

In [16]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [17]:
show_random_elements(raw_datasets["train"])

,Unnamed: 0,claim,subquestions
0,1028,Could the Port of Baltimore handle the entire world's cargo production of ginger each year?,How much cargo does the Port of Baltimore handle each fiscal quarter?\nHow much ginger cargo is produced each year?\nIs #1 greater than or equal to #2?
1,1106,Is Christopher Nolan indebted to Bob Kane?,Who created the Batman?\nTo what films was Christopher Nolan's Hollywood success attributed to from the 2000s to 2010s?\nDid #1 provide the source material to #2?
2,1806,Was King Kong (2005 film) solvent?,What does it mean to be solvent in business/finance?\nWhat was the budget (liabilities) of the 2005 movie King Kong?\nHow much did 2005 movie King Kong gross (assets) worldwide?\nDoes #3 compare favorably with #2 as defined in #1?
3,1286,Can you carry a Chrysler in a laptop bag?,"What kind of object is ""Chrysler"" referring to?\nHow much do #1's typically weigh?\nWhat object is a laptop bag designed to carry?\nHow much does #3 typically weigh?\nIs #4 greater than or equal to #2?"
4,1820,Are people banned from entering the Forbidden City?,Is the Forbidden City a tourist attraction?\nAre tourist attractions open to the public?\nAre the answers to #1 and #2 the same?



- Classify type of claim

For each type of claim
- Decompose claim
- Get claim prediction

## Tokenization and Dataset Preparation

In [18]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-base', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>'

In [19]:
max_input_length = 64
max_target_length = 128

In [20]:

def preprocess_function(examples, prefix="decompose the compositional question:"):
    inputs = [prefix + doc for doc in examples["claim"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # The "labels" are the tokenized outputs:
    print(examples["subquestions"])
    labels = tokenizer(text_target=examples["subquestions"], max_length=max_target_length, truncation=True)
    print(labels)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [21]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True, remove_columns=raw_datasets['train'].column_names)

Map:   0%|          | 0/1854 [00:00<?, ? examples/s]

['How many kids did Julius Caesar have?\nHow many kids did Genghis Khan have?\nIs #2 greater than #1?', 'Who can perform lawful arrests?\nAre members of The Police also #1?', 'Depression is caused by low levels of what chemicals?\nMonoamine Oxidase has an effect on what chemicals?\nOf the chemicals listed in both #1 and #2, does Monoamine Oxidase raise their levels?', "How sensitive is a grey seal's hearing on land?\nHow sensitive is a dog's hearing on land?\nIs #2 better than #1?", 'What is the value of the Pound Sterling based on?\nIs #1 the material used in making it?', 'What protein is Shrimp scampi made out of?\nWhat have #1 been found to contain?\nAre #2 free from plastic?', 'What country broadcasts Rede Globo?\nWhat is the official language of #1?\nIs #2 Chinese?', 'How much does a Boeing 737 cost?\nHow much did the 2017 movie Wonder Woman gross?\nIs #2 greater than #1?', 'What kind of products does Casio manufacture?\nWhat kind of products does Petco sell?\nDoes #1 overlap with

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

['What is the population of Albany, Georgia?\nWhat is the population of Albany, New York?\nWhat is the difference between 100,000 and #1?\nWhat is the difference between 100,000 and #2?\nIs #3 smaller than #4?', 'What language is used in Saint Vincent and the Grenadines?\nIs #1 based in English?', 'Is greed a  deadly sin?\nIs gluttonly a deadly sin?\nWhat percent of crimes involved greed?\nWhat percent of people are overweight?\nIf #1 and #2 are both yes is #3 a higher percentager than #4?', 'How tall is Mount Fuji?\nWhat is the maximum depth of the Sea of Japan?\nIs #1 greater than #2?', "What is Lil Jon's top ranked Billboard song?\nWhat artists contributed to #1?\nWho makes up the group The Lox?\nIs any element of #3 also an element of #2?", 'What state is Miami located in?\nWhich states are part of the American West Coast?\nIs #1 included in #2?', 'What is the size of the Swiss Guard?\nWhat is the seating capacity of the Virginia General Assembly?\nIs #1 equal to or greater than #2

Map:   0%|          | 0/207 [00:00<?, ? examples/s]

["What was Theodor Geisel's pen name?\nWhich books authored by #1 made him famous?\nAre any of #2 particularly related to the color green?", "What is Ariana Grande's signature top?\nWhat is Ariana Grande's signature shoewear?\nWhat type of clothing is #1 considered?\nIs #3 considered a comfort and item and is #2 considered a high style item?", "How many years did Richard III live to be?\nWho was Richard III's father?\nHow old did #2 live to be?\nIs #3 greater than #1?", 'What type of professional would handle bluetooth problems?\nAre dentists trained in #1?', 'When did Walt Disney pass away?\nWhen was Anderson Cooper born?\nIs #2 before #1?', 'What seasons can you expect see frost?\nWhat months do college commencements occur?\nDo any of #2 occur during #1?', 'How many children does Eddie Murphy have?\nHow many players are on a basketball team?\nHow many teams are on the basketball court at the same time?\nHow much is #2 multiplied by #3?\nIs #1 greater than or equal to #4?', 'On what c

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1854
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 229
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 207
    })
})

# Building the model

## Metrics

In [23]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    print(predictions[0])
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    print("decoded gen",decoded_preds)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value for key, value in result.items()}#{key: value['f1'] * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Loading the model

In [24]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_PATH)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [25]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

## Training Args

In [26]:
epochs = 36
batch_size = 8
lr = 2e-5

In [27]:
!pip install accelerate -U

In [28]:
args = Seq2SeqTrainingArguments(
    output_dir=SAVE_PATH,
    learning_rate=lr,
    do_train = True,
    do_eval = True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    load_best_model_at_end=True,
    num_train_epochs=epochs,
    predict_with_generate=True,
    generation_max_length=512,
    logging_dir=LOGGING_PATH,
    logging_steps=300,
    save_steps=300,
    report_to = "wandb",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [29]:
wandb_run = wandb.init(
    project="flant5_subq",
    config={
        "per_device_train_batch_size": batch_size,
        "learning_rate": lr})

now = datetime.datetime.now()
current_time = now.strftime("%m/%d/%Y, %H:%M:%S")
wandb_run.name = "run_" + current_time

wandb: Currently logged in as: szymon13gol. Use `wandb login --relogin` to force relogin


In [30]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [31]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)

<ipython-input-31-82a9305c052e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Training

In [32]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [33]:
trainer.evaluate() # Evaluation before fine-tuning

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


[    0    37 27119    16  5664    19     3     9     3   324    18 13029
   740    28     3     9  3902    13     3     5     1     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


{'eval_loss': 2.6313672065734863,
 'eval_model_preparation_time': 0.0067,
 'eval_rouge1': 0.143,
 'eval_rouge2': 0.0529,
 'eval_rougeL': 0.1156,
 'eval_rougeLsum': 0.1302,
 'eval_gen_len': 79.8079,
 'eval_runtime': 200.2108,
 'eval_samples_per_second': 1.144,
 'eval_steps_per_second': 0.145}

In [34]:
val

,Unnamed: 0,claim,subquestions
0,0,Will the Albany in Georgia reach a hundred tho...,"What is the population of Albany, Georgia?\nWh..."
1,1,Is the language used in Saint Vincent and the ...,What language is used in Saint Vincent and the...
2,2,Is greed the most prevalent of the Seven Deadl...,Is greed a deadly sin?\nIs gluttonly a deadly...
3,3,Would the top of Mount Fuji stick out of the S...,How tall is Mount Fuji?\nWhat is the maximum d...
4,4,Was Lil Jon's top ranked Billboard song a coll...,What is Lil Jon's top ranked Billboard song?\n...
...,...,...,...
224,224,Are there bones in an anchovy pizza?,Which fishes are used in anchovy pizza?\nAre #...
225,225,Can you hide a basketball in a sand cat's ear?,"On average, how large is a sand cat's ear?\nWh..."
226,226,Did Ivan the Terrible's father and grandfather...,Who was Ivan the Terrible's father?\nWho was t...
227,227,Are amoebas safe from breast cancer?,What tissue does breast cancer affect?\nIs hav...


In [35]:
# %%wandb

trainer.train()

Step,Training Loss,Validation Loss,Model Preparation Time,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
300,1.993900,1.631341,0.006700,0.456300,0.174900,0.358300,0.426800,29.751100
600,1.778700,1.592811,0.006700,0.460700,0.177100,0.363200,0.427000,30.218300
900,1.700000,1.568079,0.006700,0.466200,0.181200,0.366000,0.432600,29.751100
1200,1.623300,1.556529,0.006700,0.462400,0.179800,0.365800,0.431300,30.056800
1500,1.562400,1.548215,0.006700,0.465300,0.184700,0.372200,0.437500,29.877700
1800,1.536500,1.542286,0.006700,0.474900,0.189400,0.373200,0.441800,30.000000
2100,1.483500,1.538884,0.006700,0.471900,0.190000,0.369700,0.439600,29.585200
2400,1.443200,1.541223,0.006700,0.475000,0.191400,0.372900,0.443200,28.842800
2700,1.414600,1.538879,0.006700,0.478100,0.195600,0.375800,0.447300,30.672500
3000,1.388800,1.537776,0.006700,0.480500,0.196000,0.376000,0.450100,29.532800


[    0   363    19     8  2074    13 27119    16  5664    58   571   186
   151   619    16  7172    58    27     7 15493  2123   145  7172    58
     1     0     0     0     0     0     0     0  -100  -100  -100  -100
  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100  -100
  -100]
decoded gen ['What is the population of Albany in Georgia? How many people live in #1? Is #2 greater than #1?', 'What language is used in Saint Vincent and the Grenadines? What language is spoken in #1? Is #2 the same as #1?', 'What are the Seven Deadly Sins? What are the Seven Deadly Sins? Is #1 the most prevalent?', 'What is the height of Mount Fuji? What is the height of the Sea of Japan? Is #1 greater than #2?', 'What band did Lil Jon play with? What band did The Lox play with? Is #1 the same as #2?', "What is Miami's population? What is the population of the US West Coast? Is #2 greater than or equal to #1?", 'What is the number of seats in the Virginia General Assembly? How many seats 

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=8352, training_loss=1.3519668359866088, metrics={'train_runtime': 2277.6817, 'train_samples_per_second': 29.303, 'train_steps_per_second': 3.667, 'total_flos': 2818295423004672.0, 'train_loss': 1.3519668359866088, 'epoch': 36.0})

In [36]:
trainer.evaluate()

[    0   571   186   151    33    16     8 27119    16  5664    58   571
   186   151    33    16     8 27119    16   368  1060    58    27     7
  7172  2123   145 15493    58     1     0     0     0     0     0     0
     0     0     0     0     0  -100  -100  -100  -100  -100  -100  -100
  -100  -100  -100  -100  -100]
decoded gen ['How many people are in the Albany in Georgia? How many people are in the Albany in New York? Is #1 greater than #2?', 'What language is spoken in Saint Vincent and the Grenadines? What language is spoken in English? Is #2 the same as #1?', 'What are the Seven Deadly Sins? What are the Seven Deadly Sins? Is #1 the same as #2?', 'What is the height of Mount Fuji? What is the height of the Sea of Japan? Is #2 greater than #1?', 'What is the top ranked Billboard song of Lil Jon? What is the name of the band that played the song "The Lox"? Is #1 the same as #2?', "What is Miami's population? What is the population of the US West Coast? Is #1 greater than #2?"

{'eval_loss': 1.5377755165100098,
 'eval_model_preparation_time': 0.0067,
 'eval_rouge1': 0.4805,
 'eval_rouge2': 0.196,
 'eval_rougeL': 0.376,
 'eval_rougeLsum': 0.4501,
 'eval_gen_len': 29.5328,
 'eval_runtime': 25.0945,
 'eval_samples_per_second': 9.126,
 'eval_steps_per_second': 1.156,
 'epoch': 36.0}

In [89]:
print(SAVE_MODEL_PATH)

/content/drive/Othercomputers/macbook_pro/fact-checking/data/custom_decomposition/comparison_strategyqa/model/


In [91]:
trainer.save_model(SAVE_MODEL_PATH)

In [38]:
wandb_run.finish()

eval/gen_len,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/model_preparation_time,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/rouge1,▁▇███████████████████████████
eval/rouge2,▁▇▇▇▇▇▇▇█████████████████████
eval/rougeL,▁▇███████████████████████████
eval/rougeLsum,▁▇▇██████████████████████████
eval/runtime,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/samples_per_second,▁█▇█▇▇▇▇█▆████████▆▇█▅▅▆▆▆▅▅█
eval/steps_per_second,▁█▇█▇▇▇▇█▆████████▆▇█▅▅▆▆▆▅▆█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇█████


In [45]:
model = AutoModelForSeq2SeqLM.from_pretrained(SAVE_MODEL_PATH)

In [46]:
model.to('cuda')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
model.device

device(type='cuda', index=0)

# Get QuanTemp Subquestions

In [70]:
def generate_output(test_samples, model):
    inputs = tokenizer(
        test_samples,
        max_length=128,
        return_tensors="pt")

    input_ids = inputs.input_ids.to(model.device)
    #print(input_ids)
    #print(tokenizer.decode(input_ids[0], skip_special_tokens=True))
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask,min_length = 64, max_length = 128, do_sample=True, top_p=0.95, top_k=50)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # print(test_samples)
    return output_str

In [49]:
from src.common import read_data, save_data

train_claims = read_data('raw_data/train_claims.json')
val_claims = read_data('raw_data/val_claims.json')
test_claims = read_data('raw_data/test_claims.json')

In [54]:
# Test on one sample
print("decompose the compositional question:"+train_claims[0]['claim'], sep="\n", end="\n\n")
output = generate_output("decompose the compositional question:"+train_claims[0]['claim'], model)

decompose the compositional question:In her budget speech, Nirmala Sitharaman claimed that the Government distributed 35,000 crore LED bulbs in the country.

decompose the compositional question:In her budget speech, Nirmala Sitharaman claimed that the Government distributed 35,000 crore LED bulbs in the country.


In [66]:
[question + '?' for question in output[0].strip().split('?') if question]

['How many bulbs did Nirmala Sitharaman claim to have distributed?',
 ' How much did Nirmala Sitharaman claim to have distributed to the government in the country?',
 ' Is #2 greater than or equal to #1?',
 ' 0 What is the difference between #1 and #2?',
 ' Is #3 greater than or equal to #3?']

In [73]:
from tqdm.auto import tqdm

def generate_for_category(claims, category):
  counter = 0
  for claim in tqdm(claims):
    if claim['taxonomy_label'] == category:
      output = generate_output("decompose the compositional question:"+claim['claim'], model)
      subquestions = [question + '?' for question in output[0].strip().split('?') if question]
      claim['subquestions'] = subquestions

  print(counter)
  return claims

# train_claims_subquestions = generate_for_category(train_claims, 'comparison')
val_claims_subquestions = generate_for_category(val_claims, 'comparison')
test_claims_subquestions = generate_for_category(test_claims, 'comparison')


  0%|          | 0/3084 [00:00<?, ?it/s]

0


  0%|          | 0/2495 [00:00<?, ?it/s]

0


In [87]:
save_data('custom_decomposition/comparison_strategyqa/train_generated.json', train_claims_subquestions)
save_data('custom_decomposition/comparison_strategyqa/val_generated.json', val_claims_subquestions)
save_data('custom_decomposition/comparison_strategyqa/test_generated.json', test_claims_subquestions)